In [ ]:
!pip install -U scikit-learn==1.3.2 xgboost==1.7.6 koreanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 29.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from tqdm import tqdm

# --- 1. 데이터 불러오기 ---
# 파일 경로는 실제 환경에 맞게 수정해주세요.
# 예: df = pd.read_csv('2020_2023_최종데이터.csv')
df = pd.read_csv('/content/drive/MyDrive/기상청/2020_2023_최종데이터.csv')

# --- 2. [수정] 기본 전처리 및 정렬 ---
if 'tm_dt' in df.columns:
    df.drop(columns=['tm_dt'], inplace=True)

# [수정됨] '구'이름을 제외하고, 오직 시간('tm') 순으로만 정렬합니다.
df.sort_values(by='tm', inplace=True)

# [수정됨] 정렬된 데이터프레임에서 y를 생성합니다.
y = df['call_count'].copy()

# (이후 코드는 모두 동일합니다)


# --- 3. 피처 엔지니어링 (날짜, 공휴일, 순환 피처) ---
df['year'] = df['tm'] // 10000
df['month'] = (df['tm'] % 10000) // 100
df['day'] = df['tm'] % 100
df['weekday'] = pd.to_datetime(df['tm'], format='%Y%m%d').dt.weekday
df['day_of_year'] = pd.to_datetime(df['tm'], format='%Y%m%d').dt.dayofyear
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
df['is_before_holiday'] = df['공휴일'].shift(-1, fill_value=0)
df['is_after_holiday'] = df['공휴일'].shift(1, fill_value=0)

encoder = LabelEncoder()
df['address_city'] = encoder.fit_transform(df['address_city'])

df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df.drop(columns=['month'], inplace=True)


# --- 4. [개선] 시계열 피처 추가 (Lag & Rolling) ---
# 이제 df가 이미 정렬되어 있으므로 바로 피처를 생성할 수 있습니다.
df['call_count_lag_1'] = df.groupby('address_gu')['call_count'].shift(1)
df['call_count_lag_7'] = df.groupby('address_gu')['call_count'].shift(7)

df['call_count_rolling_mean_7'] = df.groupby('address_gu')['call_count'].shift(1).rolling(window=7, min_periods=1).mean()
df['call_count_rolling_std_7'] = df.groupby('address_gu')['call_count'].shift(1).rolling(window=7, min_periods=1).std()

df.fillna(0, inplace=True)


# --- 5. [수정] 학습/검증 데이터 분리 (데이터 누수 방지 1단계) ---
# 중요: 시계열 데이터이므로 시간 순서를 유지하기 위해 shuffle=False 옵션을 사용합니다.
# 타겟 인코딩을 하기 전에 데이터를 먼저 분리합니다.
X_train_full, X_val_full, y_train, y_val = train_test_split(
    df.drop(columns=['call_count']), y, test_size=0.2, random_state=42, shuffle=False
)


# --- 6. [수정] 타겟 인코딩 (데이터 누수 방지 2단계) ---
# [수정됨] X_train_full의 'address_gu'를 기준으로 y_train의 평균을 계산합니다.
gu_mean_map = y_train.groupby(X_train_full['address_gu']).mean()
sub_address_mean_map = y_train.groupby(X_train_full['sub_address']).mean()

# 위에서 계산된 평균값을 학습 및 검증 데이터에 모두 적용(매핑)합니다.
X_train_full['address_gu_mean_target'] = X_train_full['address_gu'].map(gu_mean_map)
X_val_full['address_gu_mean_target'] = X_val_full['address_gu'].map(gu_mean_map)

X_train_full['sub_address_mean_target'] = X_train_full['sub_address'].map(sub_address_mean_map)
X_val_full['sub_address_mean_target'] = X_val_full['sub_address'].map(sub_address_mean_map)

# 검증 데이터에만 존재하는 새로운 gu/sub_address가 있을 경우를 대비해, 전체 학습 데이터의 평균으로 결측치를 채웁니다.
overall_train_mean = y_train.mean()
X_train_full.fillna(overall_train_mean, inplace=True)
X_val_full.fillna(overall_train_mean, inplace=True)

# 위에서 계산된 평균값을 학습 및 검증 데이터에 모두 적용(매핑)합니다.
X_train_full['address_gu_mean_target'] = X_train_full['address_gu'].map(gu_mean_map)
X_val_full['address_gu_mean_target'] = X_val_full['address_gu'].map(gu_mean_map)

X_train_full['sub_address_mean_target'] = X_train_full['sub_address'].map(sub_address_mean_map)
X_val_full['sub_address_mean_target'] = X_val_full['sub_address'].map(sub_address_mean_map)

# 검증 데이터에만 존재하는 새로운 gu/sub_address가 있을 경우를 대비해, 전체 학습 데이터의 평균으로 결측치를 채웁니다.
overall_train_mean = y_train.mean()
X_train_full.fillna(overall_train_mean, inplace=True)
X_val_full.fillna(overall_train_mean, inplace=True)


# --- 7. 최종 학습 데이터 준비 ---
# 모델링에 사용할 gu 정보를 따로 저장합니다.
gu_train = X_train_full['address_gu']
gu_val = X_val_full['address_gu']

# 모델 학습에 불필요한 컬럼들을 정의하고 제거합니다.
drop_cols = ['tm', 'address_gu', 'sub_address']
X_train = X_train_full.drop(columns=drop_cols)
X_val = X_val_full.drop(columns=drop_cols)

# 메모리 효율성을 위해 데이터 타입을 float32로 변환합니다.
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
y_train = y_train.astype('float32')
y_val = y_val.astype('float32')

print("데이터 준비 완료. 학습을 시작합니다.")
print(f"학습 데이터 크기: {X_train.shape}")
print(f"검증 데이터 크기: {X_val.shape}")


# --- 8. [핵심] 구별 모델 학습 ---
# 전체 데이터로 일반 패턴을 학습하고, 각 구별 검증 데이터로 최적점을 찾는 방식
gu_models = {}
gu_scores = {}

gu_list = gu_train.unique()

for gu in tqdm(gu_list, desc="구별 모델 학습 진행"):
    # 현재 'gu'에 해당하는 검증 데이터의 인덱스를 찾습니다.
    val_gu_indices = (gu_val == gu)
    X_val_gu = X_val[val_gu_indices]
    y_val_gu = y_val[val_gu_indices]

    # 해당 'gu'의 데이터가 너무 적으면 학습을 건너뜁니다.
    if len(X_val_gu) < 10:
        print(f"⚠️ [{gu}] 검증 데이터 수 부족({len(X_val_gu)}개)으로 학습 생략")
        continue

    # XGBoost 모델 정의
    # early_stopping_rounds를 사용하므로 n_estimators는 넉넉하게 설정합니다.
    model = XGBRegressor(
        n_estimators=1000,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method='hist',
        random_state=42,
        early_stopping_rounds=50  # 50 라운드 동안 성능 개선이 없으면 조기 종료
    )

    # [중요] 전체 학습 데이터로 모델을 학습시키고,
    # 검증은 해당 'gu'의 데이터로 수행하여 조기 종료를 결정합니다.
    model.fit(
        X_train, y_train,
        eval_set=[(X_val_gu, y_val_gu)],
        verbose=False
    )

    # 해당 'gu'의 검증 데이터로 예측 및 성능 평가
    y_pred_val = model.predict(X_val_gu)
    rmse = np.sqrt(mean_squared_error(y_val_gu, y_pred_val))

    gu_models[gu] = model
    gu_scores[gu] = rmse

    print(f"✅ [{gu}] RMSE: {rmse:.4f} (최적 반복 횟수: {model.best_iteration})")


# --- 9. 전체 검증 성능 최종 평가 ---
all_preds = []
all_targets = []

# 저장된 모델을 사용하여 각 'gu'별로 예측을 수행하고 전체 결과 취합
for gu, model in gu_models.items():
    val_gu_indices = (gu_val == gu)
    X_val_gu = X_val[val_gu_indices]
    y_val_gu = y_val[val_gu_indices]

    if len(X_val_gu) > 0:
        y_pred_val = model.predict(X_val_gu)
        all_preds.extend(y_pred_val)
        all_targets.extend(y_val_gu)

# 전체 검증 데이터에 대한 최종 RMSE 계산
total_rmse = np.sqrt(mean_squared_error(all_targets, all_preds))
print(f"\n📊 전체 검증 데이터 최종 RMSE (call_count 기준): {total_rmse:.4f}")

데이터 준비 완료. 학습을 시작합니다.
학습 데이터 크기: (34339, 54)
검증 데이터 크기: (8585, 54)


구별 모델 학습 진행:   6%|▋         | 1/16 [00:07<01:47,  7.18s/it]

✅ [강서구] RMSE: 0.7974 (최적 반복 횟수: 56)


구별 모델 학습 진행:  12%|█▎        | 2/16 [00:10<01:06,  4.77s/it]

✅ [사상구] RMSE: 0.9144 (최적 반복 횟수: 57)


...

✅ [연제구] RMSE: 2.9669 (최적 반복 횟수: 109)


구별 모델 학습 진행:  94%|█████████▍| 15/16 [00:43<00:02,  2.03s/it]

✅ [해운대구] RMSE: 1.1331 (최적 반복 횟수: 38)


구별 모델 학습 진행: 100%|██████████| 16/16 [00:44<00:00,  2.80s/it]

✅ [중구] RMSE: 0.5701 (최적 반복 횟수: 55)

📊 전체 검증 데이터 최종 RMSE (call_count 기준): 1.1144


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# --- [사전 준비] ---
# 학습 과정에서 사용된 객체들이 메모리에 있다고 가정합니다.
# 만약 다른 스크립트라면, 학습 시 아래 객체들을 저장(joblib, pickle)하고 여기서 불러와야 합니다.
# 1. gu_models (학습된 모델 딕셔너리)
# 2. encoder (LabelEncoder 객체)
# 3. X.columns (학습에 사용된 최종 컬럼 순서)

# [수정] 데이터 누수 방지를 위해 학습 데이터(train set)만으로 계산된 맵을 사용해야 합니다.
# (이전 학습 코드의 6번 단계에서 생성된 맵을 사용)
# gu_mean_map = y_train.groupby(X_train_full['address_gu']).mean()
# sub_address_mean_map = y_train.groupby(X_train_full['sub_address']).mean()
# overall_train_mean = y_train.mean()

# --- 1. 데이터 로드 ---
# df는 학습에 사용된 2020-2023 데이터, valid_df는 예측할 2024년 데이터
# df = pd.read_csv('2020_2023_최종데이터.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/기상청/2024_최종데이터.csv')

# --- 2. [핵심] 과거 데이터와 예측할 데이터 합치기 ---
# Lag, Rolling 피처를 끊김 없이 계산하기 위함
combined_df = pd.concat([df, valid_df], ignore_index=True)

print(valid_df[valid_df['hm_min'] == -99].shape)

# --- 3. [핵심] 통합 피처 엔지니어링 ---
# 합쳐진 전체 데이터에 대해, 학습 때와 '완벽히 동일한' 방식으로 피처 생성

# 3-1. 시간순 정렬
combined_df.sort_values(by=['address_gu', 'tm'], inplace=True)

# 3-2. 날짜/공휴일/순환 피처 생성
combined_df['year'] = combined_df['tm'] // 10000
combined_df['month'] = (combined_df['tm'] % 10000) // 100
combined_df['day'] = combined_df['tm'] % 100
combined_df['weekday'] = pd.to_datetime(combined_df['tm'], format='%Y%m%d').dt.weekday
combined_df['day_of_year'] = pd.to_datetime(combined_df['tm'], format='%Y%m%d').dt.dayofyear
combined_df['is_weekend'] = combined_df['weekday'].isin([5, 6]).astype(int)
combined_df['is_before_holiday'] = combined_df['공휴일'].shift(-1, fill_value=0)
combined_df['is_after_holiday'] = combined_df['공휴일'].shift(1, fill_value=0)

combined_df['month_sin'] = np.sin(2 * np.pi * combined_df['month'] / 12)
combined_df['month_cos'] = np.cos(2 * np.pi * combined_df['month'] / 12)
combined_df.drop(columns=['month'], inplace=True)

print(combined_df[combined_df['hm_min'] == -99].shape)

# 3-3. Lag / Rolling 피처 생성
combined_df['call_count_lag_1'] = combined_df.groupby('address_gu')['call_count'].shift(1)
combined_df['call_count_lag_7'] = combined_df.groupby('address_gu')['call_count'].shift(7)
combined_df['call_count_rolling_mean_7'] = combined_df.groupby('address_gu')['call_count'].shift(1).rolling(window=7, min_periods=1).mean()
combined_df['call_count_rolling_std_7'] = combined_df.groupby('address_gu')['call_count'].shift(1).rolling(window=7, min_periods=1).std()

# --- 3. [핵심] 통합 피처 엔지니어링 ---
# ... (앞부분 동일) ...

# 3-4. 결측치 처리
cols_to_fill = ['call_count_lag_1', 'call_count_lag_7', 'call_count_rolling_mean_7', 'call_count_rolling_std_7']
for col in cols_to_fill:
    if col in combined_df.columns:
        # [수정] .fillna(0, inplace=True) 대신 아래 방식으로 수정 (FutureWarning 방지)
        combined_df[col] = combined_df[col].fillna(0)


# --- 4. 예측할 데이터 다시 분리 및 최종 피처 정리 ---
# 피처 엔지니어링이 완료되었으므로, 다시 2024년 데이터만 추출
pred_df = combined_df[combined_df['tm'] >= 20240101].copy()

# 4-1. 범주형 인코딩
# 학습 때 사용한 encoder를 그대로 사용
if 'Unknown' not in encoder.classes_:
    encoder.classes_ = np.append(encoder.classes_, 'Unknown')
pred_df['address_city'] = pred_df['address_city'].apply(lambda x: x if x in encoder.classes_ else 'Unknown')
pred_df['address_city'] = encoder.transform(pred_df['address_city'])

# 4-2. 타겟 인코딩
# 학습 때 만들어둔 맵을 그대로 사용
pred_df['address_gu_mean_target'] = pred_df['address_gu'].map(gu_mean_map).fillna(overall_train_mean)
pred_df['sub_address_mean_target'] = pred_df['sub_address'].map(sub_address_mean_map).fillna(overall_train_mean)

# --- 5. 예측 수행 ---
result_df_list = []

# [수정] 모호한 X.columns 대신, 학습에 사용된 것이 확실한 X_train.columns를 사용합니다.
# 만약 학습 스크립트와 예측 스크립트가 분리되어 있다면, 학습 시 X_train.columns를 파일로 저장하고
# 예측 시 불러와서 사용해야 합니다.
model_columns = X_train.columns

# [수정] drop과 컬럼 정렬을 한 번에 처리하여 실수를 방지합니다.
drop_cols_for_pred = ['tm', 'call_count', 'address_gu', 'sub_address']
X_pred_full = pred_df.drop(columns=drop_cols_for_pred, errors='ignore')

# [수정] 학습에 사용된 컬럼(model_columns)만, 동일한 순서로 가져옵니다.
X_pred_full = X_pred_full[model_columns].astype('float32')

for gu in tqdm(pred_df['address_gu'].unique(), desc="구별 예측 진행"):
    if gu not in gu_models:
        print(f"⚠️ {gu} 모델 없음 → 예측 생략")
        continue

    # 예측할 데이터에서 현재 'gu'에 해당하는 부분만 필터링
    gu_indices = (pred_df['address_gu'] == gu)
    X_pred_gu = X_pred_full[gu_indices]

    if X_pred_gu.empty:
        continue

    # 예측 (이제 이 부분에서 오류가 발생하지 않습니다)
    preds = np.round(gu_models[gu].predict(X_pred_gu)).astype(int)
    preds[preds < 0] = 0 # 혹시 모를 음수 예측 방지

    # ... (이후 코드는 동일)

    # 결과 저장을 위해 원본 데이터에 예측값 추가
    temp_df = pred_df[gu_indices].copy()
    temp_df['call_count'] = preds
    result_df_list.append(temp_df[['tm', 'address_city', 'address_gu', 'sub_address', 'call_count']])


# --- 6. 제출 파일 생성 ---
valid_preds_df = pd.concat(result_df_list, ignore_index=True)
valid_preds_df.rename(columns={'tm': 'TM'}, inplace=True)

submission_template = pd.read_csv('/content/drive/MyDrive/기상청/test_call119.csv', encoding='euc-kr')
submission_template.drop(columns=['call_count'], inplace=True, errors='ignore')

# address_city도 submission_template 기준으로 인코딩
submission_template['address_city'] = submission_template['address_city'].apply(lambda x: x if x in encoder.classes_ else 'Unknown')
submission_template['address_city'] = encoder.transform(submission_template['address_city'])


submission = submission_template.merge(
    valid_preds_df,
    on=['TM', 'address_city', 'address_gu', 'sub_address'],
    how='left'
)

submission['call_count'] = submission['call_count'].fillna(0).astype(int)
submission.to_csv('250028.csv', index=False, encoding='euc-kr')

print("✅ 제출 파일 저장 완료: 250028.csv")
print("\n# 예측 분포 확인")
print(submission['call_count'].value_counts().sort_index().tail(10)) # 상위 10개

(53, 44)
(53, 57)


구별 예측 진행: 100%|██████████| 16/16 [00:00<00:00, 98.45it/s]


✅ 제출 파일 저장 완료: 250028.csv

# 예측 분포 확인
call_count
1    2918
2    5661
3     770
4      32
5      77
6     100
7      34
8       6
9       3
Name: count, dtype: int64


In [ ]:
valid_df.tail(50)

,tm,address_city,address_gu,sub_address,stn,ta_max,ta_min,ta_max_min,hm_min,hm_max,...,부상,공휴일,평균 풍속(m/s),풍정합(100m),평균 현지기압(hPa),최고 해면기압(hPa),최저 해면기압(hPa),평균 해면기압(hPa),tm_dt,평균 지면온도(°C)
9551,20241030,부산광역시,동래구,온천동,940,23.2,10.4,12.8,36.5,92.1,...,3,0,2.4,2062,1013.3,1024.1,1019.4,1021.6,2024-10-30,17.1
9552,20241030,부산광역시,동래구,수안동,940,23.2,10.4,12.8,36.5,92.1,...,3,0,2.4,2062,1013.3,1024.1,1019.4,1021.6,2024-10-30,17.1
9553,20241030,부산광역시,부산진구,부전동,938,22.8,11.1,11.7,42.2,92.8,...,0,0,2.4,2062,1013.3,1024.1,1019.4,1021.6,2024-10-30,17.1
9554,20241031,부산광역시,북구,만덕동,941,23.3,13.0,10.3,50.4,88.6,...,1,0,2.9,2530,1014.2,1024.0,1020.9,1022.4,2024-10-31,17.5
9555,20241031,부산광역시,북구,화명동,941,23.3,13.0,10.3,50.4,88.6,...,1,0,2.9,2530,1014.2,1024.0,1020.9,1022.4,2024-10-31,17.5
9556,20241031,부산광역시,사상구,감전동,904,23.2,13.7,9.5,52.8,91.4,...,2,0,2.9,2530,1014.2,1024.0,1020.9,1022.4,2024-10-31,17.5
9557,20241031,부산광역시,사상구,덕포동,904,23.2,13.7,9.5,52.8,91.4,...,2,0,2.9,2530,1014.2,1024.0,1020.9,1022.4,2024-10-31,17.5
9558,20241031,부산광역시,사상구,모라동,904,23.2,13.7,9.5,52.8,91.4,...,2,0,2.9,2530,1014.2,1024.0,1020.9,1022.4,2024-10-31,17.5
9559,20241031,부산광역시,사상구,삼락동,904,23.2,13.7,9.5,52.8,91.4,...,2,0,2.9,2530,1014.2,1024.0,1020.9,1022.4,2024-10-31,17.5
9560,20241031,부산광역시,사상구,주례동,938,21.9,13.5,8.4,53.3,86.0,...,2,0,2.9,2530,1014.2,1024.0,1020.9,1022.4,2024-10-31,17.5


In [ ]:
submission['call_count'].value_counts()

,count
call_count,
2,5661
1,2918
3,770
6,100
5,77
7,34
4,32
8,6
9,3
